# CNJCx Week 5: Practical Python

Tyler Benster
(tbenst@stanford.edu)

## Outline
### Motivation and background
### Hands-on coding

### Motivation and background
- Goals
- Anti-goals
- Extra details
- Tidy Data
- Today's Python Packages

## Goals for today
- "Day in the life" of a Pythonista
- Whirlwhind tour of foudational packages for Data Scientists in Python
- Exposure to opinionated best-practices for formating data and code
- understand the "why" of each code block
- know which library to use for particular analyses

## Anti-goals for today
- comprehend the "how" of each line of code
- know which function to use for particular analyses
- understand the math behind shown analyses
- feeling that the class is going at a comfortable pace
- understand how this presentation was made in a Jupyter notebook with RISE/reveal.js

## Extra details for eager or advanced listeners
- <details>
    <summary><a><strong>IYI</strong></a>: If You're Interested; click me! (no seriously please do :)</summary>
    Optional content will be prefaced by IYI. This is not essential for understanding the presentation, and if you are at all feeling lost or confused, now is a great time to ignore what I'm saying and ask questions in the chat. IYI is inspired by David Foster Wallace's Infinite Jest.
</details>
- Bonus: quick peak at modern deep learning in Pytorch

## Easy visualization with Tidy Data
![tidy data](https://r4ds.had.co.nz/images/tidy-1.png)

See Hadley Wickham's [publication](https://www.jstatsoft.org/article/view/v059i10) for more details and motivation.

### Hands-on coding
- Data visualization: how to make some basic plots (matplotlib, Altair)
- (5 minute break)
- Advanced data analysis: interrogate the data and visualize(scipy.stats, sklearn)
- how to read in common data formats (csv)
- data munging: what data structures and patterns to use for optimal efficiency (numpy, pytorch tensor, pandas, tidy data)

### Get this notebook running
1. open a terminal
2. navigate to folder where you want the code, e.g. `mkdir -p ~/code && cd code`
3. `git clone https://github.com/tbenst/cnjcx-course-materials.git`

4. `cd cnjcx-course-materials`
5. `git checkout week5`
6. Activate your environment
7. `pip install -r requirements.txt`
8. `jupyter notebook`

## First-up: matplotlib
matplotlib is the most popular plotting library in Python, and is a swiss army knife that can do virtually anything. It's also the most involved to use.

Let's load some example data first

In [ ]:
from sklearn import datasets
import matplotlib.pyplot as plt
import numpy as np

# load data
iris = datasets.load_iris()
iris.keys()

In [ ]:
for key, value in iris.items():
    if not key in ['data', 'target']:
        print(f"=========\n{key}: {value}")

Let's create a basic scatter plot using the procedural (scripting) interface

In [ ]:
plt.scatter(iris.data[:,0], iris.data[:,2])
plt.xlabel(iris.feature_names[0])
plt.ylabel(iris.feature_names[2])

Now, we create subplots with coloring & legend using the alternate Object-oriented interface

In [ ]:
fig, axes = plt.subplots(nrows=1,ncols=2)
axes[0].scatter(iris.data[:,2], iris.data[:,3], c=iris.target)
axes[0].set_xlabel(iris.feature_names[2])
axes[0].set_ylabel(iris.feature_names[3])
scatter1 = axes[1].scatter(iris.data[:,0], iris.data[:,1], c=iris.target)
axes[1].set_xlabel(iris.feature_names[0])
axes[1].set_ylabel(iris.feature_names[1])
axes[1].legend(scatter1.legend_elements()[0],
               iris.target_names, title="Species")

Uh oh, that looks terrible. Here's a quick fix:

In [ ]:
fig.tight_layout()
fig

Better, but legend location still problematic.

**IYI**: This can be fixed using low-level arguments like `bbox`, see [here](https://stackoverflow.com/questions/4700614/how-to-put-the-legend-out-of-the-plot)

## Surely there's a better way??
Introducing the "Grammar of Graphics"! Other python GoG packages include Seaborn and Holoviews. We use **Altair**, as it is implemented on the cross-language Vega-lite, so what you learn today can also be done in Julia or even used for interactive web-charts!

![grammar of graphics](https://miro.medium.com/max/2000/1*mcLnnVdHNg-ikDbHJfHDNA.png)

**IYI** conceptual guide [here](https://towardsdatascience.com/a-comprehensive-guide-to-the-grammar-of-graphics-for-effective-visualization-of-multi-dimensional-1f92b4ed4149)

## Introducing pandas: convient tables in python

First, let's install a python package with example datasets

In [ ]:
!pip install vega_datasets

Next we load an example DataFrame

In [ ]:
from vega_datasets import data
import altair as alt, pandas as pd

cars_df = data.cars()
print(f"object type: {type(cars_df)}")

DataFrames have some convenient methods to help us inspect it

In [ ]:
cars_df.head(2)

In [ ]:
cars_df.columns

In [ ]:
cars_df.tail(1)

Some of these methods can be chained:

In [ ]:
cars_df.Name.tail()

Here we select a single value

In [ ]:
cars_df["Name"][402]

Let's take a look at the type of each Series (column)

In [ ]:
cars_df.dtypes

Let's see the various Origins

In [ ]:
cars_df.Origin.unique()

We can easily do `where` queries

In [ ]:
cars_df[cars_df.Origin=='USA'].head()

Or chain multiple requirements

In [ ]:
np.all?

In [ ]:
from datetime import datetime
idxs = np.all([cars_df.Origin=='USA',
              cars_df.Horsepower>200,
              cars_df.Year<=datetime(1972,1,1)],
             axis=0)
cars_df[idxs]

# Plotting Tidy Data with Altair
Since our data is Tidy, we can use the Grammar of Graphics to make plots!

In [ ]:
line = alt.Chart(cars_df).mark_line().encode(
    x='Year',
    y='mean(Miles_per_Gallon)'
)
# https://altair-viz.github.io/user_guide/generated/core/altair.ErrorBandDef.html#altair.ErrorBandDef
band = alt.Chart(cars_df).mark_errorband(extent='ci').encode(
    x='Year',
    y=alt.Y('Miles_per_Gallon', title='Miles/Gallon'),
)

band + line

The power of this approach becomes especially apparent with complex plots that would require a lot of work for each axis with matplotlib

In [ ]:
line = alt.Chart(cars_df).mark_line().encode(
    x='Year',
    y=alt.Y('mean(Miles_per_Gallon)', title="average MPG"),
    color='Cylinders:O' # we specify that the data is Ordinal, meaning ordered
).properties(
    width=180,
    height=180
).facet(
    facet='Origin:N', # data is Nominal, meaning categorical
    columns=3
)
line

### Excercise 1: make a scatter plot of Horsepower vs Acceleration, colored by Origin
Instead of `mark_line`, use `mark_point`

In [ ]:
# your code here...feel free to refer to cells above!

Let's quickly revist the Iris dataset and show off our new skills!

In [ ]:
iris_df = data.iris()

alt.Chart(iris_df).mark_circle().encode(
    alt.X('sepalLength', scale=alt.Scale(zero=False)),
    alt.Y('sepalWidth', scale=alt.Scale(zero=False, padding=1)),
    color='species',
    size='petalWidth'
)

Finally, **IYI**, here's a more advanced figure: an interactive scatter & Violin plot using `selection`, `transform_filter`, and `transform_density`

In [ ]:
brush = alt.selection(type='interval', resolve='global')
scatter = alt.Chart(cars_df).mark_point().encode(
    x=alt.X('Horsepower'),
    y=alt.Y('Acceleration'),
    color=alt.condition(brush, 'Origin', alt.ColorValue('gray'))
)

violin = alt.Chart(cars_df).transform_filter(
    brush
).transform_density(
    'Miles_per_Gallon',
    as_=['Miles_per_Gallon', 'density'],
    extent=[5, 50],
    groupby=['Origin']
).mark_area(orient='horizontal').encode(
    y='Miles_per_Gallon:Q',
    color='Origin:N',
    x=alt.X(
        'density:Q',
        stack='center',
        impute=None,
        title=None,
        axis=alt.Axis(labels=False, values=[0],grid=False, ticks=True),
    ),
    column=alt.Column(
        'Origin:N',
        header=alt.Header(
            titleOrient='bottom',
            labelOrient='bottom',
            labelPadding=0,
        ),
    )
).properties(
    width=100
)


plot = (scatter | violin).add_selection(
# scatter.add_selection(
    brush
).configure_facet(
     spacing=0
).configure_view(
    stroke=None
)

In [ ]:
# try drawing a box on the scatter plot!B
plot

For more, checkout this example gallery of beautiful plots with shockingly few lines of code: https://altair-viz.github.io/gallery/index.html

## (5 minute break)

**IYI** A poem while we wait

In [ ]:
import this

## Data munging

Suppose you get data from a collaborator. How might you read a csv and convert to Tidy Data?

We'll look at some data collected by Darwin Babino @ University of Washington. Each row contains the trial-summed response to a 0.5s flash of light

In [ ]:
# Read in csv file to DataFrame
rgcs_df = pd.read_csv("rgc_light_response.csv")

# Each column with number is a 1ms time bin that sums
# the number of Action potentials from `ntrials`.
# i, j index the 2D electrode array.
# unit_num identifies puported individual neurons recorded from each electrode.
rgcs_df.head(2)

In [ ]:
# Note: if you're coming from Matlab, you might be tempted to write a for loop
# sometimes this can't be avoided, but in general writing for-loops is discouraged
# in Python. They are slow in interpreted languages and usually there's a better way

rgcs_tidy = pd.melt(rgcs_df, id_vars=['retina', 'id', 'ntrials'],
        var_name="time_bin",
        value_name="spike_count",
        value_vars=list(map(str, np.arange(3500))))
rgcs_tidy.head(1)

Normalize spike count by number of trials

In [ ]:
# 1ms time bins
time_bin = 1000
rgcs_tidy["time"] = (pd.to_numeric(rgcs_tidy.time_bin) + 1) / time_bin
rgcs_tidy["firing_rate"] = rgcs_tidy.spike_count / rgcs_tidy.ntrials * time_bin
rgcs_tidy.drop(columns=["spike_count", "ntrials", "time_bin"], inplace=True)
rgcs_tidy

In [ ]:
# limitation with Altair...very slow with lots of rows!
# This plot will work if you uncomment the next line
# but may be slow, so don't try this right now :)

# alt.data_transformers.disable_max_rows()

line = alt.Chart(rgcs_tidy).mark_line(
    stroke="black"
).encode(
    x="time:Q",
    y="mean(firing_rate)",
#     detail="firing_rate"
)

darkness = pd.DataFrame({"start": [0.0,1.5], "end":[1.0, 3.5]})

# See Vega-lite docuentation on rect mark
# https://vega.github.io/vega/docs/marks/rect/
rect = alt.Chart(darkness).mark_rect(
    fillOpacity=0.1,
    fill='black'
).encode(
    x='start',
    x2='end',
)


line+rect


Instead, let's prepare to plot with matplotlib. First, we create a nSamples x nFeatures matrix

In [ ]:
# effectively, this undoes our melt operation
rgcs_tidy.pivot(index='id', columns='time', values='firing_rate').head()

In [ ]:
# convert to ndarray
rgc_mat = np.array(rgcs_tidy.pivot(index='id', columns='time', values='firing_rate'))

In [ ]:
from matplotlib import patches
from typing import Tuple
time = np.arange(rgc_mat.shape[1])/1000 # convert to seconds
fig, ax = plt.subplots()
ax.plot(time, rgc_mat.mean(0))
ylim = ax.get_ylim()
# Create a Rectangle patch

def make_rect(start:float, duration:float, ylim:Tuple[float, float]):
    return patches.Rectangle((start, ylim[0]), duration, ylim[1],
                             facecolor='black', alpha=0.1)

# we make small helper function to follow DRY: Don't repeat yourself
rect1 = make_rect(0,1, ylim)
rect2 = make_rect(1.5,2, ylim)

# Add the patch to the Axes
ax.add_patch(rect1)
ax.add_patch(rect2)
ax.set_xlim(0,3.5)

## Calculating instantaneous firing rate
$\omega(\tau) = \frac{1}{\sqrt{2\pi}\sigma_\omega}\exp\left({-\frac{\tau^2}{2\sigma_\omega^2}}\right)$

(to type this in markdown):
```
$\omega(\tau) = \frac{1}{\sqrt{2\pi}\sigma_\omega}\exp\left({-\frac{\tau^2}{2\sigma_\omega^2}}\right)$
```

**IYI**: see 1.11 from [Dayan and Abbott](http://www.gatsby.ucl.ac.uk/~lmate/biblio/dayanabbott.pdf)

In [ ]:
from scipy import signal
# estimate firing rate using gaussian smoothing
sigma = 6
bandwidth = 0.05 # sec
bin_width = 0.001
transformed_sigma = bandwidth/bin_width
window = signal.gaussian(2*sigma*transformed_sigma,
                         std=transformed_sigma)[None]

# instantaneous firing rate (acausal)
ifr = signal.convolve(rgc_mat, window,mode="same") \
             / (transformed_sigma*np.sqrt(2*np.pi))

In [ ]:
def plot_rgc_trace(ax, trace, time=time,
                   light_on=1, light_off=1.5):
    ax.plot(time,trace)
    ylim = ax.get_ylim()
    rect1 = make_rect(0,1, ylim)
    rect2 = make_rect(1.5,2, ylim)
    ax.add_patch(rect1)
    ax.add_patch(rect2)
    ax.set_xlim(0,3.5)
    ax.set_xlabel("time (s)")
    ax.set_ylabel("firing rate (Hz)")

fig, axes = plt.subplots(5,2, figsize=(8,8))
plot_rgc_trace(axes[0,0], rgc_mat[13])
axes[0,0].set_title("Trial-average firing rate")
axes[0,1].set_title("Instantaneous firing rate")
plot_rgc_trace(axes[0,1], ifr[13])

for i,c in zip(range(1,6),[33, 23, 19, 21]):
    plot_rgc_trace(axes[i,0], rgc_mat[c])
    plot_rgc_trace(axes[i,1], ifr[c])

In [ ]:
fig

## Statistical analysis
Now that we have experience with some data munging and visualization, let's try our hand at a quick statistical analysis using **scipy**!

Question: In Iris dataset, does sepal length vary in a statistically significant way across the three species?


In [ ]:
# let's take a quick look at the data for intuition
alt.Chart(iris_df).mark_bar().encode(
    x=alt.X("sepalWidth", bin=alt.Bin(step=0.25)),
    y="count(sepalWidth)",
    color="species"
)

We run an ANOVA

In [ ]:
import scipy.stats as stats
species = iris_df.species.unique()
data = [iris_df.sepalWidth[iris_df.species == s] for s in species]
F, p = stats.f_oneway(*data)
# We reject the null
F, p

## sklearn: a unified interface to machine learning
As long as your data can be organized into a nSamples x nFeatures matrix and fits in RAM, you can use sklearn!

Let's try our hand at dimensionality reduction

In [ ]:
from sklearn.decomposition import PCA

# turn species into vector of integers
species_vector = iris_df.species.astype('category').cat.codes

# drop species & create data matrix
data = np.array(iris_df.iloc[:,:-1])

pca = PCA(n_components=2)
projected_data = pca.fit_transform(data)

plt.scatter(projected_data[:,0], projected_data[:,1],
           c = species_vector)

# IYI: try adding a legend!

In [ ]:
from sklearn import cluster 
kmeans = cluster.KMeans(n_clusters=3)
kmeans.fit(data)

plt.hist(kmeans.labels_, bins=np.arange(kmeans.labels_.max()+2))
plt.title("Count by cluster label")

In [ ]:
from sklearn.manifold import TSNE
plt.figure()
plt.subplot(121)
plt.scatter(projected_data[:,0], projected_data[:,1],
           c = species_vector)
plt.title("True labels")
plt.subplot(122)
plt.scatter(projected_data[:,0], projected_data[:,1],
           c = kmeans.labels_)
plt.title("Clustered labels")

In [ ]:
tsne = TSNE(n_components=2)
tsne_data = tsne.fit_transform(data)

plt.scatter(tsne_data[:,0], tsne_data[:,1],
           c = species_vector)

## Bonus: Variational auto-encoder in PyTorch

This download may take a while, so feel free to just watch :)

![mnist](https://upload.wikimedia.org/wikipedia/commons/2/27/MnistExamples.png)

In [ ]:
!pip install torch torchvision
!mkdir -p mnist

In [ ]:
import torch
from torchvision import datasets, transforms

from torch import nn, optim, utils
from torchvision import datasets, transforms
from torch.nn import functional as F
import torchvision.utils
from functools import partial

device = torch.device("cpu")
torch.manual_seed(20200909) # reproducible analysis

batch_size = 64
kwargs = {'num_workers': 1, 'pin_memory': True}
mnist_train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('./mnist', train=True, download=True,
                   transform=transforms.ToTensor()),
    batch_size=batch_size, shuffle=True, **kwargs)
mnist_test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('./mnist', train=False, transform=transforms.ToTensor()),
    batch_size=batch_size, shuffle=True, **kwargs)

# shape is batch_size x 1 x 28 x 28

## Now witness the firepower of this fully armed and operational Notebook!

**IYI**: see a full presentation [here](https://github.com/tbenst/cnjc-vae/blob/master/main.pdf)

In [ ]:
# subclass PyTorch Module for reverse-mode autodifferentiation 
# for easy backpropogation of loss gradient
class VAE(nn.Module):
    
    def __init__(self, nfeatures,nlatent=20):
        super(VAE, self).__init__()
        self.nfeatures = nfeatures
        self.nhidden = int(nfeatures/5)
        
        # nn.Linear is a "dense" layer of form y = Ax + b
        
        # Encoder layers
        self.hidden_encoder = nn.Linear(nfeatures, self.nhidden)
        # mean encoding layer 
        self.mean_encoder = nn.Linear(self.nhidden, nlatent)
        # log variance encoding layer 
        self.logvar_encoder = nn.Linear(self.nhidden, nlatent)
        
        # Decoder layers
        self.hidden_decoder = nn.Linear(nlatent, int(nfeatures/5))
        self.reconstruction_decoder = nn.Linear(self.nhidden, nfeatures)

    def encode(self, x):
        # we use a ReLu (rectified linear unit) activation function
        h1 = F.relu(self.hidden_encoder(x))
        return self.mean_encoder(h1), self.logvar_encoder(h1)

    def reparameterize(self, mean, logvar):
        """Reparameterize out stochastic node so the gradient can propogate 
           deterministically."""

        if self.training:
            standard_deviation = torch.exp(0.5*logvar)
            # sample from unit gaussian with same shape as standard_deviation
            epsilon = torch.randn_like(standard_deviation)
            return epsilon * standard_deviation + mean
        else:
            return mean

    def decode(self, z):
        h3 = F.relu(self.hidden_decoder(z))
        # use sigmoid to bound output to (0,1)
        return F.sigmoid(self.reconstruction_decoder(h3))

    
    def forward(self, x):
        "A special method in PyTorch modules that is called by __call__"
        
        # flatten batch x height x width into batch x nFeatures, then encode
        mean, logvar = self.encode(x.view(-1, self.nfeatures))
        # sample an embedding, z
        z = self.reparameterize(mean, logvar)
        # return the (sampled) reconstruction, mean, and log variance
        return self.decode(z), mean, logvar
    
def loss_function(recon_x, x, mu, logvar, nfeatures):
    "Reconstruction + KL divergence losses summed over all elements and batch."
    BCE = F.binary_cross_entropy(recon_x, x.view(-1, nfeatures), size_average=False)

    # we want KLD = - 0.5 * sum(1 + log(sigma^2) - mu^2 - sigma^2)
    # where sigma is standard deviation and mu is mean
    # (interested? check out Appendix B of https://arxiv.org/abs/1312.6114)
    # In pytorch, x^2 is written as x.pow(2), e^x is written as x.exp(),
    # and sum_{i=1}^n (x_i + y_i) for x,y of length n
    # can be written as torch.sum(x+y)
    KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
    return BCE + KLD

def train(epoch, model, optimizer, train_loader, log_interval=10):
    model.train()
    train_loss = 0
    for batch_idx, data in enumerate(train_loader):
        data = data[0].to(device)  # we ignore any labels & transfer to GPU
        nfeatures = data[0].numel()
        optimizer.zero_grad()
        recon_batch, mu, logvar = model(data)
        loss = loss_function(recon_batch, data, mu, logvar, nfeatures)
        loss.backward()
        train_loss += loss.item()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader),
                loss.item() / len(data)))

    print('====> Epoch: {} Average loss: {:.4f}'.format(
          epoch, train_loss / len(train_loader.dataset)))


def test(epoch, model, test_loader,folder="results"):
    model.eval()
    test_loss = 0
    with torch.no_grad():
        for i, data in enumerate(test_loader):
            data = data[0].to(device)
            nfeatures = data[0].numel()
            n = min(data.size(0), 15)
            if len(data.shape)==3:
                  # zebrafish
                _, H, W = data.shape
                dat = data[:n,None]
            elif len(data.shape)==4:
                  # MNIST
                _, _, H, W = data.shape
                dat = data[:n]
            recon_batch, mu, logvar = model(data)
            test_loss += loss_function(recon_batch, data, mu, logvar, nfeatures).item()
            if i == 0:              
                comparison = torch.cat([dat,
                                   recon_batch.view(-1, 1, H, W)[:n]])
                torchvision.utils.save_image(comparison.cpu(),
                         folder+'/reconstruction_' + str(epoch) + '.png', nrow=n)

    test_loss /= len(test_loader.dataset)
    print('====> Test set loss: {:.4f}'.format(test_loss))

In [ ]:
# run cell to reset model
nfeatures = 28**2
# we use a latent space of dimension 2 as to get an easy-to-visualize manifold
# (see mnist/sample_*.png while running next cell)
nlatent = 2
mnist_model = VAE(nfeatures,nlatent=nlatent).to(device)
mnist_optimizer = optim.Adam(mnist_model.parameters(), lr=1e-3)

In [ ]:
# this will take two minutes to run.
# As it does, check out the mnist folder!
# each epoch, reconstruction examples are saved (original on top) 
# select files on right, then refresh, and double click image
# click bottom right corner of image to resize

nepochs = 5
H, W = (28,28)

# make grid of z1 x z2 where z1,z2 \elem (-3.5,-2.5, ..., 3.5)
nrow = 25
latents = torch.zeros(nrow,nrow,nlatent)
z1_tick = np.linspace(-3.5,3.5,nrow)
z2_tick = np.linspace(-3.5,3.5,nrow)
for i, z1 in enumerate(z1_tick):
    for j, z2 in enumerate(z2_tick):
        latents[i,j,[0,1]] = torch.FloatTensor([z1,z2])
latents = latents.to(device)

for epoch in range(1, nepochs + 1):
    train(epoch, mnist_model, mnist_optimizer, mnist_train_loader)
    test(epoch, mnist_model, mnist_test_loader,folder='mnist')
    with torch.no_grad():
        latent_space = mnist_model.decode(latents.view(-1,nlatent)).cpu()
        torchvision.utils.save_image(latent_space.view(-1, 1, H, W),
                   'mnist/sample_' + str(epoch) + '.png',nrow=nrow)

Thanks for tuning in!